- [x] 测试是否有等于0的数据 -> 无
- [x] 测试增加 > <后，sys.maximize之后是否正确 -> 查看column等变量没有问题
- [x] 测试@符号与x[value].sum()之间是否相等
- [x] 优化fun的for循环
- [x] 优化query constraints里的itertool.product方法的for循环
- [ ] 减少列数
- [ ] 修改sys.maximize为一个合适的大小，之后再对数据进行normalization，其实在LPALG中不需要normalization，这点在神经网络里就显得特别重要了，先变成normal distribution 再求逆映射转换回原先的分布。normal distribution能加速网络学习，提升网络拟合精度。
- [ ] 尝试multiprocessing、joblib
- [ ] 尝试pulp、OSQP（C++）

In [5]:
# LPALG
import sys
import time
import argparse
import itertools
import numpy as np
import pandas as pd

import datasets
import estimators as estimators_lib

# import scipy as sc
from scipy import optimize
from copy import deepcopy

# from scipy.sparse import csr_matrix

In [6]:
def Oracle(table, query):
    cols, idxs, ops, vals = query
    oracle_est = estimators_lib.Oracle(table)
    return oracle_est.Query(cols, ops, vals)

In [7]:
def cal_true_card(query, table):
    cols, idxs, ops, vals = query
    ops = np.array(ops)
    probs = Oracle(table, (cols, idxs, ops, vals))
    return probs

In [8]:
def GenerateQuery(table, min_num_filters, max_num_filters, rng, dataset):
    """Generate a random query."""
    num_filters = rng.randint(max_num_filters - 1, max_num_filters)
    cols, idxs, ops, vals = SampleTupleThenRandom(table, num_filters, rng, dataset)
    sel = cal_true_card((cols, idxs, ops, vals), table) / float(table.cardinality)
    return cols, idxs, ops, vals, sel

In [9]:
def SampleTupleThenRandom(table, num_filters, rng, dataset):
    vals = []
    new_table = table.data
    s = new_table.iloc[rng.randint(0, new_table.shape[0])]
    vals = s.values
    if dataset in ["dmv", "dmv-tiny", "order_line"]:
        vals[6] = vals[6].to_datetime64()
    elif dataset in ["orders1", "orders"]:
        vals[4] = vals[4].to_datetime64()
    elif dataset == "lineitem":
        vals[10] = vals[10].to_datetime64()
        vals[11] = vals[11].to_datetime64()
        vals[12] = vals[12].to_datetime64()
    idxs = rng.choice(len(table.columns), replace=False, size=num_filters)
    cols = np.take(table.columns, idxs)
    # If dom size >= 10, okay to place a range filter.
    # Otherwise, low domain size columns should be queried with equality.
    ops = rng.choice(["<", ">", "<=", ">=", "="], size=num_filters)
    ops_all_eqs = ["="] * num_filters
    sensible_to_do_range = [c.DistributionSize() >= 10 for c in cols]
    ops = np.where(sensible_to_do_range, ops, ops_all_eqs)
    # if num_filters == len(table.columns):
    #     return table.columns,np.arange(len(table.columns)), ops, vals
    vals = vals[idxs]
    op_a = []
    val_a = []
    for i in range(len(vals)):
        val_a.append([vals[i]])
        op_a.append([ops[i]])
    return cols, idxs, pd.DataFrame(op_a).values, pd.DataFrame(val_a).values

In [10]:
def dictionary_column_interval(table_size, query_set):
    # Traverse all queries to apply the intervalization skill for each column
    n_column = table_size[1]
    column_interval = {}
    for i in range(n_column):
        column_interval[i] = set(
            [0, sys.maxsize]
        )  # use set([0, sys.maxsize]) to adapt '>' and '<'.
    for query in query_set:
        col_idxs = query[1]
        vals = query[3]
        for i in range(len(col_idxs)):
            column_interval[col_idxs[i]].add(vals[i][0])
    for k, v in column_interval.items():
        if not v:
            column_interval[k] = [0]
        else:
            column_interval[k] = sorted(list(v))
    return column_interval

In [11]:
def dictionary_column_variable(column_to_interval):
    # Assign a sequential index to each interval in each column
    column_to_variable = {}
    total_intervals = 0  # count how many intervals in total
    column_variable_number = []  # count how many intervals in each column
    for k, v in column_to_interval.items():
        count = len(v)
        column_to_variable[k] = [total_intervals + i for i in range(count)]
        total_intervals += count
        column_variable_number.append(count)
    return total_intervals, column_variable_number, column_to_variable

In [12]:
def dictionary_variable_interval(column_to_interval, column_to_variable):
    # Map each interval index to the left endpoint of its corresponding interval
    variable_to_interval = {}
    for column, variable in column_to_variable.items():
        for i in range(len(variable)):
            variable_to_interval[variable[i]] = column_to_interval[column][i]
    return variable_to_interval

In [13]:
def op_to_variables(column_to_variable, column, index, op):
    # Find all matching intervals' index in the column based on the operator and input interval index
    column_interval_idx = np.array(column_to_variable[column])
    if op == ">":
        return list(column_interval_idx[index + 1 :])
    elif op == ">=":
        return list(column_interval_idx[index:])
    elif op == "=":
        return [column_interval_idx[index]]
    elif op == "<":
        return list(column_interval_idx[:index])
    elif op == "<=":
        return list(column_interval_idx[: index + 1])

In [14]:
def dictionary_query_to_interval(query_set, column_to_interval, column_to_variable):
    # Traverse all queries to find their corresponding interval index
    query_to_interval = {}
    for i in range(len(query_set)):
        col_idxs = query_set[i][1]
        ops = query_set[i][2]
        vals = query_set[i][3]
        query_to_interval[i] = []
        for j in range(len(col_idxs)):
            column = col_idxs[j]
            index = column_to_interval[column].index(vals[j][0])
            query_to_interval[i].append(
                op_to_variables(column_to_variable, column, index, ops[j][0])
            )
    return query_to_interval

In [15]:
def dictionary_query_to_x_index(query_set, query_to_interval, column_to_variable):
    # Traverse all queries to find their corresponding x index
    query_to_x_index = {}
    for i in range(len(query_set)):
        col_idxs = query_set[i][1]
        query_to_x_index[i] = [[] for i in range(n_column)]
        for j in range(len(col_idxs)):
            column = col_idxs[j]
            variable = query_to_interval[i][j]
            for k in variable:
                x_index = column_to_variable[column].index(k)
                query_to_x_index[i][column].append(x_index)
    return query_to_x_index

In [16]:
def transfer_x_index(query_to_x_index, column_variable_number):
    # Transfer all empty x-index-list to all-indexes-list corresponding to the column
    x_index = {}
    for k, v in query_to_x_index.items():
        x_index[k] = []
        for i in range(len(v)):
            if v[i] == []:
                x_index[k].append([j for j in range(column_variable_number[i])])
            else:
                x_index[k].append(v[i])
    return x_index

In [17]:
# Build Minimize Problem

In [18]:
def x0():
    return np.ones(total_x) * n_row / total_x

In [19]:
def bounds():
    return np.array([[0, n_row]] * total_x)

In [20]:
def constraints():
    return [{"type": "eq", "fun": lambda x: n_row - x.sum()}]

In [21]:
def query_constraints(query_set, x_index, column_variable_number):
    query_constraints_list = []
    #     find = np.array([
    #         np.product(column_variable_number[i:])
    #         for i in range(1, len(column_variable_number))
    #     ] + [1])
    flip = np.flip(np.array(column_variable_number + [1]))
    find = np.flip(flip.cumprod())[1:]

    for key, values in x_index.items():
        sel = query_set[key][4]
        x_ind = np.array([x for x in itertools.product(*values)])  # , dtype=np.uint16)
        result = np.dot(x_ind, find)  # same as x_ind @ find

        def value_constraints(x, sel=sel, value=result):
            return x[value].sum() - sel * n_row

        #         row = np.zeros(len(result))
        #         col = result
        #         data = np.ones(len(result))
        #         matrix = csr_matrix((data, (row, col)), shape=(1, total_x))#, dtype=np.uint16)
        #         def value_constraints(x, sel=sel, matrix=matrix):
        #             return matrix @ x - sel * n_row

        query_constraints_list.append(value_constraints)
    return query_constraints_list

In [22]:
def fun():
    def error(x):
        return sum([constraint(x) ** 2 for constraint in query_constraints_list]) / (n_row)  # **2

    return error

In [23]:
def randomized_rouding(x):
    int_x = deepcopy(x)
    for i in range(len(x)):
        xi = x[i]
        floor = np.floor(xi)
        ceil = np.ceil(xi)
        if not floor == ceil:
            int_x[i] = np.random.choice([floor, ceil], p=[xi - floor, ceil - xi])
    return int_x

In [24]:
def generate_table_data(column_to_interval, int_x, column_variable_number):
    df = pd.DataFrame(
        columns=[f"col_{i}" for i in range(n_column)], index=[i for i in range(int_x.sum())]
    )

    column_to_x = []
    for i in column_variable_number:
        column_to_x.append([j for j in range(i)])
    all_x = np.array([x for x in itertools.product(*column_to_x)], dtype=np.uint16)

    count = 0
    for i in range(total_x):  # Here: total_x == len(int_x), n_column == all_x.shape[1]
        if int_x[i] != 0:
            df.iloc[count : count + int_x[i], :] = [
                column_to_interval[j][all_x[i][j]] for j in range(n_column)
            ]
            count += int_x[i]
    return df

In [25]:
def execute_query(our_table, query_set):
    diff = []
    for query in query_set:
        sentence = ""
        for i in range(len(query[0])):
            if i != 0:
                sentence += " and "
            sentence += f"col_{query[1][i]}"
            if query[2][i][0] == "=":
                sentence += "=="
            else:
                sentence += query[2][i][0]
            sentence += f"{query[3][i][0]}"
        sel = our_table.query(sentence).shape[0] / our_table.shape[0]
        sel2 = query[4]  # round(query[4] * n_row)
        if sel == 0:
            print("oh, zero sel!")
            sel += 1 / our_table.shape[0]
        if sel2 == 0:
            print("oh, zero sel2!")
            sel2 += 1 / n_row
        if sel < sel2:
            diff.append(sel2 / sel)
        else:
            diff.append(sel / sel2)
    return diff

In [26]:
# if __name__ == '__main__':

In [27]:
parser = argparse.ArgumentParser()
parser.add_argument("--dataset", type=str, default="wine", help="Dataset.")
parser.add_argument("--query-size", type=int, default=5, help="query size")
parser.add_argument("--reload", type=bool, default=False, help="reload")
parser.add_argument("--num-conditions", type=int, default=2, help="num of conditions")

# args = parser.parse_args()  # for python
args, unknown = parser.parse_known_args()  # for jupyter notebook

In [28]:
# assert args.dataset in ['dmv-tiny', 'dmv']
if args.dataset == "dmv-tiny":
    table = datasets.LoadDmv("dmv-tiny.csv")
elif args.dataset == "dmv":
    table = datasets.LoadDmv()
else:
    type_casts = {}
    if args.dataset in ["orders1"]:
        type_casts = {4: np.datetime64, 5: np.float}
    elif args.dataset in ["orders"]:
        type_casts = {4: np.datetime64}
    elif args.dataset == "lineitem":
        type_casts = {10: np.datetime64, 11: np.datetime64, 12: np.datetime64}
    elif args.dataset == "order_line":
        type_casts = {6: np.datetime64}

    table = datasets.LoadDataset(args.dataset + ".csv", args.dataset, type_casts=type_casts)

if not args.reload:
    print("Begin Generating Queries ...")
    rng = np.random.RandomState(1234)
    query_set = [
        GenerateQuery(table, 2, args.num_conditions + 1, rng, args.dataset)
        for i in range(args.query_size)
    ]
    print("Complete Generating Queries.")

load dataset wine.csv done
(6497, 13)
(6497, 13)
0
1
2
3
4
5
6
7
8
9
10
11
12
0 106
1 187
2 89
3 316
4 214
5 135
6 276
7 975
8 108
9 111
10 111
11 7
12 2
Begin Generating Queries ...
Complete Generating Queries.


In [29]:
# LPALG
print("\n\nBegin LPALG...")
tic = time.time()
table_size = table.data.shape
n_row = table_size[0]
n_column = table_size[1]
column_to_interval = dictionary_column_interval(table_size, query_set)
total_intervals, column_variable_number, column_to_variable = dictionary_column_variable(
    column_to_interval
)
variable_to_interval = dictionary_variable_interval(column_to_interval, column_to_variable)
total_x = np.product(column_variable_number)
query_to_interval = dictionary_query_to_interval(query_set, column_to_interval, column_to_variable)
query_to_x_index = dictionary_query_to_x_index(query_set, query_to_interval, column_to_variable)
x_index = transfer_x_index(query_to_x_index, column_variable_number)
query_constraints_list = query_constraints(query_set, x_index, column_variable_number)

print(f"\n Solving LP problem with total param = {total_x} ...")
res = optimize.minimize(
    fun(),
    x0(),
    method="SLSQP",
    constraints=constraints(),
    bounds=bounds(),
    # tol=1e-323,
    # options={'maxiter': 1e8},
    # options={'maxiter': 1},
)
print("\n Optimize.minimize Solver Status: \n", res)
int_x = randomized_rouding(res.x).astype(int)
print(f"\n Integer X: ( length = {len(int_x)} )\n", int_x)


# generate data
dataNew = generate_table_data(column_to_interval, int_x, column_variable_number)
# print(dataNew)


# calculate error
diff = execute_query(dataNew, query_set)
print(
    f"\n\n Q-error of LPALG (query size={args.query_size}, condition={args.num_conditions}, total param={total_x}):\n"
)
print(f"min:    {np.min(diff)}")
print(f"10:     {np.percentile(diff, 10)}")
print(f"20:     {np.percentile(diff, 20)}")
print(f"30:     {np.percentile(diff, 30)}")
print(f"40:     {np.percentile(diff, 40)}")
print(f"median: {np.median(diff)}")
print(f"60:     {np.percentile(diff, 60)}")
print(f"70:     {np.percentile(diff, 70)}")
print(f"80:     {np.percentile(diff, 80)}")
print(f"mean:   {np.mean(diff)}")
print(f"90:     {np.percentile(diff, 90)}")
print(f"max:    {np.max(diff)}")
toc = time.time()
total_time = toc - tic
m, s = divmod(total_time, 60)
h, m = divmod(m, 60)
print(f"Time passed:  {h:0>2.0f}:{m:0>2.0f}:{s:0>2.0f}")



Begin LPALG...

 Solving LP problem with total param = 373248 ...


KeyboardInterrupt: 

# Pulp

In [196]:
import pulp

In [ ]:
MyProbLP = pulp.LpProblem(
    "LPProbDemo1", sense=pulp.LpMaximize
)  # 定义问题，求最小值 pulp.LpMinimize / 最大值

In [204]:
x1 = pulp.LpVariable("x1", lowBound=0, upBound=7, cat="Integer")
x2 = pulp.LpVariable("x2", lowBound=0, upBound=7, cat="Integer")
x3 = pulp.LpVariable("x3", lowBound=0, upBound=7, cat="Continuous")

In [205]:
MyProbLP += 2 * x1 + 3 * x2 - 5 * x3  # 设置目标函数
MyProbLP += 2 * x1 - 5 * x2 + x3 >= 10  # 不等式约束
MyProbLP += x1 + 3 * x2 + x3 <= 12  # 不等式约束
MyProbLP += x1 + x2 + x3 == 7  # 等式约束

In [206]:
MyProbLP.solve()

1

In [210]:
print(MyProbLP)  # 输出问题设定参数和条件

LPProbDemo1:
MAXIMIZE
2*x1 + 3*x2 + -5*x3 + 0
SUBJECT TO
_C1: 2 x1 - 5 x2 + x3 >= 10

_C2: x1 + 3 x2 + x3 <= 12

_C3: x1 + x2 + x3 = 7

VARIABLES
0 <= x1 <= 7 Integer
0 <= x2 <= 7 Integer
x3 <= 7 Continuous



In [207]:
print("Status:", pulp.LpStatus[MyProbLP.status])  # 输出求解状态

Status: Optimal


In [208]:
for v in MyProbLP.variables():
    print(v.name, "=", v.varValue)  # 输出每个变量的最优值

x1 = 7.0
x2 = 0.0
x3 = 0.0


In [209]:
print("F(x) = ", pulp.value(MyProbLP.objective))  # 输出最优解的目标函数值

F(x) =  14.0


In [ ]:
# 第二个实例

In [69]:
import pulp

# 1. 建立问题
AlloyModel = pulp.LpProblem("Data Gen", pulp.LpMinimize)
# 2. 建立变量
material = [i for i in range(7)]
mass = pulp.LpVariable.dicts("x", material, lowBound=0, upBound=n_row, cat="Continuous")
# 3. 设置目标函数
cost = {0: 16, 1: 10, 2: 8, 3: 9, 4: 48, 5: 60, 6: 53}
AlloyModel += pulp.lpSum([cost[item] * mass[item] for item in material]), "Query error"
# # 4. 施加约束
AlloyModel += pulp.lpSum([mass[item] for item in material]) == n_row, "table length"
# 5. 求解
AlloyModel.solve()
# 6. 打印结果
print(AlloyModel)  # 输出问题设定参数和条件
print("优化状态:", pulp.LpStatus[AlloyModel.status])
for v in AlloyModel.variables():
    print(v.name, "=", v.varValue)
print("最优总成本 = ", pulp.value(AlloyModel.objective))

Data_Gen:
MINIMIZE
16*x_0 + 10*x_1 + 8*x_2 + 9*x_3 + 48*x_4 + 60*x_5 + 53*x_6 + 0
SUBJECT TO
table_length: x_0 + x_1 + x_2 + x_3 + x_4 + x_5 + x_6 = 6497

VARIABLES
x_0 <= 6497 Continuous
x_1 <= 6497 Continuous
x_2 <= 6497 Continuous
x_3 <= 6497 Continuous
x_4 <= 6497 Continuous
x_5 <= 6497 Continuous
x_6 <= 6497 Continuous

优化状态: Optimal
x_0 = 0.0
x_1 = 0.0
x_2 = 6497.0
x_3 = 0.0
x_4 = 0.0
x_5 = 0.0
x_6 = 0.0
最优总成本 =  51976.0


/Users/hubble/opt/anaconda3/lib/python3.7/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


## many attributes

In [233]:
import pulp

In [267]:
# 1. 建立最小化问题
AlloyModel = pulp.LpProblem("many_attributes", pulp.LpMinimize)

In [268]:
x1 = pulp.LpVariable("x1", lowBound=0, upBound=7, cat="Integer")
x2 = pulp.LpVariable("x2", lowBound=0, upBound=7, cat="Integer")
x3 = pulp.LpVariable("x3", lowBound=0, upBound=7, cat="Continuous")
x4 = pulp.LpVariable("x4", lowBound=0, upBound=7, cat="Continuous")

In [269]:
# 3. 设置目标函数
L = np.array([[1, 1, 1, 1], [0, 1, 1, 0], [0, 0, 1, 1]])
b = np.array([50, 30, 40])


AlloyModel += x1 + x2 + x3 + x4  # pulp.lpSum([item**2 for item in x1]), "总生产成本"

In [ ]:
# # 4. 施加约束
carbonPercent = {"废料1": 0.8, "废料2": 0.7, "废料3": 0.85, "废料4": 0.4, "镍": 0, "铬": 0, "钼": 0}
NiPercent = {"废料1": 18, "废料2": 3.2, "废料3": 0, "废料4": 0, "镍": 100, "铬": 0, "钼": 0}
CrPercent = {"废料1": 12, "废料2": 1.1, "废料3": 0, "废料4": 0, "镍": 0, "铬": 100, "钼": 0}
MoPercent = {"废料1": 0, "废料2": 0.1, "废料3": 0, "废料4": 0, "镍": 0, "铬": 0, "钼": 100}
AlloyModel += pulp.lpSum([mass[item] for item in material]) == 1000, "质量约束"
AlloyModel += (
    pulp.lpSum([carbonPercent[item] * mass[item] for item in material]) >= 0.65 * 1000,
    "碳最小占比",
)
AlloyModel += (
    pulp.lpSum([carbonPercent[item] * mass[item] for item in material]) <= 0.75 * 1000,
    "碳最大占比",
)
AlloyModel += (
    pulp.lpSum([NiPercent[item] * mass[item] for item in material]) >= 3.0 * 1000,
    "镍最小占比",
)
AlloyModel += (
    pulp.lpSum([NiPercent[item] * mass[item] for item in material]) <= 3.5 * 1000,
    "镍最大占比",
)
AlloyModel += (
    pulp.lpSum([CrPercent[item] * mass[item] for item in material]) >= 1.0 * 1000,
    "铬最小占比",
)
AlloyModel += (
    pulp.lpSum([CrPercent[item] * mass[item] for item in material]) <= 1.2 * 1000,
    "铬最大占比",
)
AlloyModel += (
    pulp.lpSum([MoPercent[item] * mass[item] for item in material]) >= 1.1 * 1000,
    "钼最小占比",
)
AlloyModel += (
    pulp.lpSum([MoPercent[item] * mass[item] for item in material]) <= 1.3 * 1000,
    "钼最大占比",
)
AlloyModel += mass["废料1"] <= 75, "废料1可用量"
AlloyModel += mass["废料2"] <= 250, "废料2可用量"

In [270]:
# 5. 求解
AlloyModel.solve()

1

In [271]:
# 6. 打印结果
print(AlloyModel)  # 输出问题设定参数和条件

many_attributes:
MINIMIZE
1*x1 + 1*x2 + 1*x3 + 1*x4 + 0
VARIABLES
0 <= x1 <= 7 Integer
0 <= x2 <= 7 Integer
x3 <= 7 Continuous
x4 <= 7 Continuous



In [272]:
print("优化状态:", pulp.LpStatus[AlloyModel.status])
for v in AlloyModel.variables():
    print(v.name, "=", v.varValue)
print("最优总成本 = ", pulp.value(AlloyModel.objective))

优化状态: Optimal
x1 = 0.0
x2 = 0.0
x3 = 0.0
x4 = 0.0
最优总成本 =  0.0
